## Bayesian Model for Predicting Player Impact Based on College and Draft Position

In [87]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd
import numpy as np
import time
import requests

In [72]:
win_share_25 = pd.read_csv('Data/NBA Win Shares Leaders 24-25.csv')
#will specify starting year of season
win_share_25['Year'] = 2024
#rearrange columns for readibility
new_order = ['Rk','Year','Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
            '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
            'USG%', 'OWS', 'DWS', 'WS▼', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
            'Awards']
win_share_25 = win_share_25[new_order]
win_share_24 = pd.read_csv('Data/NBA Win Shares Leaders 23-24.csv')
win_share_24['Year'] = 2023
win_share_24 = win_share_24[new_order]
win_share_23 = pd.read_csv('Data/NBA Win Shares Leaders 22-23.csv')
win_share_23['Year'] = 2022
win_share_23 = win_share_23[new_order]
win_share_22 = pd.read_csv('Data/NBA Win Shares Leaders 21-22.csv')
win_share_22['Year'] = 2021
win_share_22 = win_share_22[new_order]
win_share_21 = pd.read_csv('Data/NBA Win Shares Leaders 20-21.csv')
win_share_21['Year'] = 2020
win_share_21 = win_share_21[new_order]

win_shares = pd.concat([win_share_25, win_share_24, win_share_23, win_share_22, win_share_21], ignore_index=True, sort=False)
win_shares.head()

,Rk,Year,Player,Age,Team,Pos,G,GS,MP,PER,...,USG%,OWS,DWS,WS▼,WS/48,OBPM,DBPM,BPM,VORP,Awards
0,1,2024,Shai Gilgeous-Alexander,26,OKC,PG,76,76,2598,30.7,...,34.8,11.9,4.8,16.7,0.309,8.9,2.6,11.5,8.9,"MVP-1,DPOY-10,CPOY-8,AS,NBA1"
1,2,2024,Nikola Jokić,29,DEN,C,70,70,2571,32.0,...,29.5,12.7,3.8,16.4,0.307,9.9,3.3,13.3,9.8,"MVP-2,CPOY-2,AS,NBA1"
2,3,2024,Ivica Zubac,27,LAC,C,80,80,2624,22.3,...,19.5,7.0,4.7,11.7,0.215,2.3,0.8,3.1,3.4,"DPOY-6,DEF2"
3,4,2024,Jarrett Allen,26,CLE,C,82,82,2296,22.1,...,15.9,7.6,4.0,11.6,0.243,3.0,1.2,4.2,3.6,NaN
4,5,2024,Giannis Antetokounmpo,30,MIL,PF,67,67,2289,30.5,...,35.2,7.8,3.7,11.5,0.241,6.9,2.5,9.5,6.6,"MVP-3,DPOY-8,AS,NBA1"


Next pull all college data and store as a dictionary

Creating a function to ensure we don't hit rate limits

In [93]:
def get_player_info(player_id, retries=3, delay=30):
    for attempt in range(retries):
        try:
            time.sleep(delay)
            return commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        except requests.exceptions.ReadTimeout:
            if attempt < retries - 1:
                time.sleep(delay*2)
            else:
                raise 

In [ ]:
active_players = win_shares['Player'].str.strip().unique()

info_dict = {}

for player in active_players:
    try:
        if player in info_dict:
            continue
        
        # 1. Try full name search
        result = players.find_players_by_full_name(player)
        
        # 2. If no full match → try partial match
        if not result:
            result = players.find_players_by_partial_name(player)
        
        # 3. If still no match → skip
        if not result:
            print("Could not find:", player)
            info_dict[player] = {'College': 'NA', 'Draft Pick': 'NA'}
            continue
        
        # 4. Extract the player_id
        curr_player_id = result[0]['id']
        
        # 5. Pull player info
        player_info = get_player_info(player_id=curr_player_id)
        
        info = player_info.common_player_info.get_data_frame()
        
        college = info.loc[0, 'SCHOOL']
        draft_pick = info.loc[0, 'DRAFT_NUMBER']
        
        info_dict[player] = {'College': college, 'Draft Pick': draft_pick}

    except Exception as e:
        print("Failed:", player, "| Reason:", e)
        info_dict[player] = {'College': 'NA', 'Draft Pick': 'NA'}

    time.sleep(15)


Failed: Shai Gilgeous-Alexander | Reason: 'DataSet' object has no attribute 'get_data_frames'


KeyboardInterrupt: 

Saving the dictionary since it took nearly 2 hrs to pull

In [96]:
test24 = win_share_24['Player'].str.strip().unique()

info_dict = {}

for player in active_players:
    try:
        if player in info_dict:
            continue
        
        # 1. Try full name search
        result = players.find_players_by_full_name(player)
        
        # 2. If no full match → try partial match
        if not result:
            result = players.find_players_by_partial_name(player)
        
        # 3. If still no match → skip
        if not result:
            print("Could not find:", player)
            info_dict[player] = {'College': 'NA', 'Draft Pick': 'NA'}
            continue
        
        # 4. Extract the player_id
        curr_player_id = result[0]['id']
        
        # 5. Pull player info
        player_info = get_player_info(player_id=curr_player_id)
        
        info = player_info.common_player_info.get_data_frame()
        
        college = info.loc[0, 'SCHOOL']
        draft_pick = info.loc[0, 'DRAFT_NUMBER']
        
        info_dict[player] = {'College': college, 'Draft Pick': draft_pick}

    except Exception as e:
        print("Failed:", player, "| Reason:", e)
        info_dict[player] = {'College': 'NA', 'Draft Pick': 'NA'}

    time.sleep(15)


Failed: A.J. Green | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Ron Holland | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Brandon Boston Jr. | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Jamal Cain | Reason: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Failed: GG Jackson II | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Xavier Tillman Sr. | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Jeenathan Williams | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: Matthew Hurt | Reason: module 'nba_api.stats.static.players' has no attribute 'find_players_by_partial_name'
Failed: B.J. Johnson | Reason: module 'nba_

In [97]:
import pickle

with open('player_info_dict.pkl', 'wb') as f:
    pickle.dump(info_dict, f)

Creating the College and Draft Number columns in the dataframe 

In [60]:
win_shares['College'] = win_shares['Player'].map(lambda x: info_dict[x]['College'])
win_shares['Draft Pick'] = win_shares['Player'].map(lambda x: info_dict[x]['Draft Pick'])

cleaned = win_shares.drop(columns=['G','GS','MP','TS%','3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS','WS/48', 'OBPM', 'DBPM','VORP', 'Awards'])

cleaned = cleaned.rename(columns={'WS▼':'WS'})
cleaned

,Rk,Year,Player,Age,Team,Pos,PER,WS,BPM,College,Draft Pick
0,1,2024,Shai Gilgeous-Alexander,26,OKC,PG,30.7,16.7,11.5,Kentucky,11
1,2,2024,Nikola Jokić,29,DEN,C,32.0,16.4,13.3,Mega Basket,41
2,3,2024,Ivica Zubac,27,LAC,C,22.3,11.7,3.1,Mega Basket,32
3,4,2024,Jarrett Allen,26,CLE,C,22.1,11.6,4.2,Texas,22
4,5,2024,Giannis Antetokounmpo,30,MIL,PF,30.5,11.5,9.5,Filathlitikos,15
...,...,...,...,...,...,...,...,...,...,...,...
2495,496,2020,Chandler Hutchison,24,2TM,SF,6.8,-0.1,-5.6,Boise State,22
2496,497,2020,Andre Roberson,29,BRK,SG,4.2,-0.1,-6.1,NA,NA
2497,498,2020,Tyrell Terry,20,DAL,PG,6.0,-0.1,-5.0,NA,NA
2498,499,2020,Jalen Lecque,20,IND,PG,4.7,-0.1,-13.0,Brewster Academy (NH),Undrafted


{'College': 'NA', 'Draft Pick': 'NA'}

In [91]:
player_info = commonplayerinfo.CommonPlayerInfo(player_id=players.find_players_by_full_name(active_players[16])[0]['id'])
info = player_info.common_player_info.get_dict()
info_df = pd.DataFrame(info['data'], columns=info['headers'])
        
college = info_df.loc[0, 'SCHOOL']
draft_pick = info_df.loc[0, 'DRAFT_NUMBER']

college, draft_pick

('Georgia', '1')

In [95]:
player_info.common_player_info.get_data_frame()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1628983,Shai,Gilgeous-Alexander,Shai Gilgeous-Alexander,"Gilgeous-Alexander, Shai",S. Gilgeous-Alexander,shai-gilgeous-alexander,1998-07-12T00:00:00,Kentucky,Canada,...,shai_gilgeous-alexander,2018,2025,N,Y,Y,2018,1,11,N
